In [1]:
# importing the required libraries

import numpy as np 
import json 
import glob

# gensim 

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


#spacy 
import spacy
from nltk.corpus import stopwords

#vis 
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


### Preparing the Data 

In [2]:
# creating a function to load and write the json data 

def load_data(file):
    with open(file, 'r',encoding='utf-8') as f:
        data= json.load(f)
    return data 


def write_data(file, data):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f)

In [3]:
# let's get the list of stopwords
stopwords = stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [4]:
# Load the data 
data = load_data(r"data\ushmm_dn.json")['texts']
print(data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [5]:
# lemmatization using spacy 
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [6]:
# generate new words 
def gen_words(texts):
    final =[]
    for text in texts:
        new= gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final
  
data_words= gen_words(lemmatized_texts)

#### Bigrams and Trigrams 

In [7]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])


['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [8]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [9]:
# # creating a dictionary of words 
# ld2word = corpora.Dictionary(data_words)

# corpus=[]
# for text in data_words:
#     new = ld2word.doc2bow(text)
#     corpus.append(new)

# print(corpus[0][0:20])

In [10]:
# word= ld2word[[0][:1][0]]

In [11]:
# word

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=30, 
                                            random_state=100,
                                            update_every=1, 
                                            chunksize=100, 
                                            passes=10, 
                                            alpha='auto')

In [13]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(1, 0.27880788), (7, 0.048639536), (9, 0.011969122), (11, 0.27440786), (15, 0.24109037), (17, 0.034882054), (19, 0.019126762), (29, 0.063026346)]
[(1, 0.27880788), (11, 0.27440786), (15, 0.24109037), (29, 0.063026346), (7, 0.048639536), (17, 0.034882054), (19, 0.019126762), (9, 0.011969122)]


In [21]:
#saving the model 
lda_model.save("models/test_model.model")


In [22]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")


In [23]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(1, 0.2788109), (7, 0.04863416), (9, 0.011969014), (11, 0.2744082), (15, 0.24109021), (17, 0.034882847), (19, 0.019126747), (29, 0.063027896)]
[(1, 0.2788109), (11, 0.2744082), (15, 0.24109021), (29, 0.063027896), (7, 0.04863416), (17, 0.034882847), (19, 0.019126747), (9, 0.011969014)]


### Visualization of the Data

In [19]:
import pyLDAvis.gensim

# Assuming you have already created your LDA model (lda_model), corpus, and id2word

pyLDAvis.enable_notebook()
viz = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=id2word, mds='mmds', R=30)
viz


d:\Projects and Work\NLP Practice\Gensim\virtual\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
21    -0.282755 -0.297603       1        1  9.583851
20     0.166351 -0.250082       2        1  8.947308
12    -0.375849 -0.194571       3        1  7.885854
22     0.307251 -0.233200       4        1  7.650814
2      0.201569 -0.348833       5        1  7.260447
1      0.411957 -0.150204       6        1  5.774334
29     0.415589  0.156024       7        1  5.693896
18     0.059552 -0.427540       8        1  5.264212
16    -0.155224 -0.267372       9        1  5.063938
7      0.014889 -0.332118      10        1  4.920490
15    -0.413950 -0.020915      11        1  4.072062
3     -0.398631  0.117482      12        1  3.496761
9     -0.257853 -0.137580      13        1  3.374697
13    -0.164374 -0.411255      14        1  3.199917
14     0.012251  0.433148      15        1  2.619571
17    -0.216240  0.405972      16        1  2.554533
5     -0.330407  0.201917      17        1  2.416607
24    -0.191520  0.056355      18        1  1.872923
28     0.026623 -0.087040      19        1  1.437160
26     0.221547 -0.042936      20        1  1.371899
10     0.370938  0.067295      21        1  1.364616
19     0.245920  0.279541      22        1  1.364475
8     -0.114959  0.291241      23        1  1.021800
0      0.076927  0.298725      24        1  0.646052
11    -0.071772  0.142703      25        1  0.613655
25     0.130238  0.219161      26        1  0.315160
4      0.125127  0.132756      27        1  0.186159
27     0.061261  0.131620      28        1  0.010879
23     0.063640  0.134485      29        1  0.010837
6      0.061904  0.132821      30        1  0.005093, topic_info=           Term         Freq        Total Category  logprob  loglift
852         use  1605.000000  1605.000000  Default  30.0000  30.0000
337      ghetto  2619.000000  2619.000000  Default  29.0000  29.0000
663    remember  1107.000000  1107.000000  Default  28.0000  28.0000
440        kill  1475.000000  1475.000000  Default  27.0000  27.0000
27         also   970.000000   970.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
751     special     0.000619   124.330244  Topic30  -9.6693  -2.3260
2033  transport     0.000619   482.721149  Topic30  -9.6693  -3.6825
543      number     0.000619   249.240251  Topic30  -9.6693  -3.0214
398    hospital     0.000619   334.071030  Topic30  -9.6693  -3.3144
65         away     0.000619   192.805009  Topic30  -9.6693  -2.7647

[1914 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         2  0.462140  able
0         4  0.055536  able
0         9  0.029752  able
0        13  0.319333  able
0        20  0.029752  able
...     ...       ...   ...
3497      5  0.091651  zone
3497     11  0.091651  zone
3497     15  0.510625  zone
3497     16  0.091651  zone
3497     23  0.065465  zone

[4083 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[22, 21, 13, 23, 3, 2, 30, 19, 17, 8, 16, 4, 10, 14, 15, 18, 6, 25, 29, 27, 11, 20, 9, 1, 12, 26, 5, 28, 24, 7])